In [ ]:
import pandas as pd
import numpy as np
import time as time
import pickle
from sklearn.utils import shuffle
import seaborn as sns
import matplotlib.pyplot as plt
import random
import networkx as nx
from sklearn.metrics import classification_report

In [ ]:
datadir = "data/cora_transductive"
max_groupsize = 169
min_groupsize = 2
num_categories = 7

In [ ]:
G = nx.read_edgelist(datadir+"/cora.cites", nodetype=str) # must be in edgelist (node1 node2\n) format

In [ ]:
classes = {}
classmap = {"Neural_Networks":"1","Case_Based":"2","Reinforcement_Learning":"3", "Probabilistic_Methods":"4", "Genetic_Algorithms":"5", "Rule_Learning":"6", "Theory": "7"}
vertex_features = {}
vertex_labels = {}
with open(datadir+"/cora.content") as f:
    content = f.readlines()
    
for line in content:
    keyval = line.split("\t")
    feature = []
    for val in keyval[1:1434]:
        if val=="0":
            feature.append(0)
        if val=="1":
            feature.append(1)
    classes[str(keyval[0])] = classmap[str(keyval[1434])[:-1]]
    
    label = np.zeros((num_categories,))
    label[int(classmap[str(keyval[1434])[:-1]])-1] = 1
    vertex_labels[str(keyval[0])] = np.asarray(label)
    vertex_features[str(keyval[0])] = np.asarray(feature)

In [ ]:
from sklearn.decomposition import PCA

feature_array = []

for feature_index in vertex_features:
    feature = vertex_features[feature_index]
    feature_array.append(feature)
pca_pre = np.asarray(feature_array)
pca = PCA(n_components = 64)
pca.fit(pca_pre)
features_new = pca.transform(pca_pre)
i = 0
for feature_index in vertex_features:
    vertex_features[feature_index] = features_new[i]
    i+=1

In [ ]:
feature_dimension = 64

def getFeaturesTrainingData():
    
    i = 0
    lists = []
    labels = []
    for vertex in G.nodes:
        vertex_embedding_list = []
        lists.append({"f":vertex_features[vertex].tolist()})
        labels.append(vertex_labels[vertex])
    X_unshuffled = []
    for hlist in lists:
        x = np.zeros((feature_dimension,))
        x[:feature_dimension] = hlist["f"]
        X_unshuffled.append(x)
    labels = np.asarray(labels)
    X_arr, Y_arr = shuffle(X_unshuffled, labels)
    X_Features = np.asarray(X_arr)
    Y_Features = np.asarray(Y_arr)
    return X_Features, Y_Features

X_Features, Y_Features = getFeaturesTrainingData()

from sklearn.model_selection import train_test_split
X_Features_train, X_Features_test, Y_Features_train, Y_Features_test = train_test_split(X_Features, Y_Features, train_size=0.05, test_size=0.95)

In [ ]:
%load_ext autoreload
%autoreload 2
from Models import FeaturesMLP
from keras.callbacks import ModelCheckpoint, TensorBoard
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
dataset_name = "cora_transductive"
batch_size = 16
num_epochs = 50

FeaturesMLP_model = FeaturesMLP(input_dimension=feature_dimension,
                 num_outputs=num_categories,
                 dataset_name=dataset_name)
plot_model(FeaturesMLP_model, to_file='images/'+dataset_name+'_FeaturesMLP_model.png')
checkpointer = ModelCheckpoint(filepath='weights/'+dataset_name+'/FeaturesMLP_weights.hdf5', verbose=0, save_best_only=True)
tbCallBack = TensorBoard(log_dir='logs/'+dataset_name+'/FeaturesMLP_logs', histogram_freq=0, batch_size=batch_size, write_graph=True, write_grads=True, write_images=True, update_freq='batch')

history = FeaturesMLP_model.fit(X_Features_train, Y_Features_train, epochs=num_epochs, batch_size=batch_size,
        shuffle=True, validation_split=0,
        callbacks=[checkpointer,tbCallBack], verbose=1)

In [ ]:
target_names = ["Neural Networks","Case Based","Reinforcement Learning","Probabilistic Methods","Genetic Algorithms","Rule Learning","Theory"]
from sklearn.metrics import classification_report, accuracy_score

Y_pred_Features = FeaturesMLP_model.predict(X_Features_test, batch_size=16, verbose=1)
finals_pred_Features = []
finals_test_Features = []
for p in Y_pred_Features:
    m = 0
    ind = 0
    final = 0
    for i in p:
        if i>m:
            m=i
            final=ind
        ind+=1
    finals_pred_Features.append(final)

for i in Y_Features_test:
    ind=0
    for j in i:
        if j==1:
            finals_test_Features.append(ind)
        ind+=1
            
print(classification_report(finals_test_Features, finals_pred_Features, target_names=target_names))
print(accuracy_score(finals_test_Features, finals_pred_Features))

In [ ]:
hyperedges = {}
nodes = G.nodes
max_size = 0
min_size = 100000
for node in nodes:
    neighborhood = list(G.neighbors(node))
    if len(neighborhood)+1 <= max_groupsize and len(neighborhood)+1 >= min_groupsize:
        hyperedges[node] = {}
        hyperedges[node]["members"] = []
        hyperedges[node]["members"].append(node)
        hyperedges[node]["category"] = classes[node]
        for neighbor in neighborhood:
            hyperedges[node]["members"].append(neighbor)
    if len(neighborhood)+1 < min_size:
        min_size = len(neighborhood)+1
    if len(neighborhood) > max_size:
        max_size = len(neighborhood)+1
print(min_size, max_size)
print(len(hyperedges))

In [ ]:
cat_amounts = {}
cat_names = {"1":"Neural Networks","2":"Case Based","3":"Reinforcement Learning", "4":"Probabilistic Methods", "5":"Genetic Algorithms", "6":"Rule Learning", "7": "Theory"}

for h in hyperedges:
    if cat_names[str(hyperedges[h]["category"])] not in cat_amounts:
        cat_amounts[cat_names[str(hyperedges[h]["category"])]]=1
    else:
        cat_amounts[cat_names[str(hyperedges[h]["category"])]]+=1

In [ ]:
vertexMemberships = {}
for h_index in hyperedges:
    hyperedge = hyperedges[h_index]
    authors = hyperedge["members"]
    for author in authors:
        if author in vertexMemberships:
            vertexMemberships[author].append(h_index)
        else:
            authorMembershipList = []
            authorMembershipList.append(h_index)
            vertexMemberships[author] = authorMembershipList

In [ ]:
from HypergraphRandomWalks import SubsampleAndTraverse

walksSaT = SubsampleAndTraverse(length=25, 
                                num_walks=50, 
                                hyperedges=hyperedges, 
                                vertexMemberships=vertexMemberships,
                                alpha=1., 
                                beta=0)

In [ ]:
from HypergraphRandomWalks import TraverseAndSelect

walksTaS = TraverseAndSelect(length=25, 
                             num_walks=50, 
                             hyperedges=hyperedges, 
                             vertexMemberships=vertexMemberships,
                             alpha=1.,
                             beta=0)

In [ ]:
from Embeddings import EmbedWord2Vec

vertex_embedding_dimension = 16
hyperedge_embedding_dimension = 128

vertex_ids, vertex_embeddings = EmbedWord2Vec(walks=walksSaT,dimension=vertex_embedding_dimension)
print("Vertex embeddings finished.")
hyperedge_ids, hyperedge_embeddings = EmbedWord2Vec(walks=walksTaS,dimension=hyperedge_embedding_dimension)
print("Hyperedge embeddings finished.")

In [ ]:
def getTrainingData():
    i = 0
    lists = []
    labels = []
    for h in hyperedges:
        vertex_embedding_list = []
        hyperedge = hyperedges[h]
        for vertex in hyperedge["members"]:
            i+=1
            if i%100000==0:
                print(i)
            try: # Good ol' nondeterminism
                vertex_embedding_list.append(vertex_embeddings[vertex_ids.index(vertex)].tolist())
            except:
                print("Missed one: ",vertex)
        lists.append({"v":vertex_embedding_list,"h":hyperedge_embeddings[hyperedge_ids.index(h)].tolist(),"f":vertex_features[h].tolist()})
        label = np.zeros((num_categories,))
        label[int(hyperedge["category"])-1] = 1
        labels.append(label)
    X_unshuffled = []
    
    for hlist in lists:
        np_vertex_embeddings = np.asarray(hlist["v"])
        x = np.zeros((hyperedge_embedding_dimension + vertex_embedding_dimension*max_groupsize + feature_dimension,))
        i = 0
        x[:hyperedge_embedding_dimension] = hlist["h"]
        x[hyperedge_embedding_dimension + vertex_embedding_dimension*max_groupsize:] = hlist["f"]
        for embedding in np_vertex_embeddings:
            x[hyperedge_embedding_dimension + i*embedding.shape[0]:hyperedge_embedding_dimension + (i+1)*embedding.shape[0]] = embedding
            i+=1
        X_unshuffled.append(x)
    labels = np.asarray(labels)
    X_arr, Y_arr = shuffle(X_unshuffled, labels)
    X = np.asarray(X_arr)
    Y = np.asarray(Y_arr)
    return X, Y

X, Y = getTrainingData()

print(X.shape)
print(Y.shape)

# np.save(datadir+'/X_hyperedge_'+str(max_groupsize)+'.npy', X)
# np.save(datadir+'/Y_hyperedge_'+str(max_groupsize)+'.npy', Y)

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.1, test_size=0.9)

print(X_train.shape)

In [ ]:
%load_ext autoreload
%autoreload 2

from Models import DeepHyperedgesTransductive
from keras.callbacks import ModelCheckpoint, TensorBoard
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

dataset_name = "cora_transductive"
batch_size = 16
num_epochs = 75

deephyperedges_transductive_model = DeepHyperedgesTransductive(vertex_embedding_dimension=vertex_embedding_dimension,
                                    hyperedge_embedding_dimension=hyperedge_embedding_dimension, 
                                    feature_dimension=feature_dimension,
                                    max_hyperedge_size=max_groupsize,
                                    num_outputs=num_categories,
                                    dataset_name=dataset_name)
plot_model(deephyperedges_transductive_model, to_file='images/'+dataset_name+'_deephyperedges_transductive_model.png')
checkpointer = ModelCheckpoint(filepath='weights/'+dataset_name+'/deephyperedges_transductive_weights.hdf5', 
                               verbose=0, save_best_only=True)
tbCallBack = TensorBoard(log_dir='logs/'+dataset_name+'/deephyperedges_transductive_logs', 
                         histogram_freq=0, batch_size=batch_size, write_graph=True, 
                         write_grads=True, write_images=True, update_freq='batch')

history = deephyperedges_transductive_model.fit(X_train, Y_train, epochs=num_epochs, batch_size=batch_size,
        shuffle=True, validation_split=0,
        callbacks=[checkpointer,tbCallBack], verbose=1)

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

target_names = ["Neural Networks","Case Based","Reinforcement Learning","Probabilistic Methods","Genetic Algorithms","Rule Learning","Theory"]

y_pred = deephyperedges_transductive_model.predict(X_test, batch_size=16, verbose=1)
finals_pred = []
finals_test = []
for p in y_pred:
    m = 0
    ind = 0
    final = 0
    for i in p:
        if i>m:
            m=i
            final=ind
        ind+=1
    finals_pred.append(final)

for i in Y_test:
    ind=0
    for j in i:
        if j==1:
            finals_test.append(ind)
        ind+=1
            
print(classification_report(finals_test, finals_pred, target_names=target_names,digits=4))
print(accuracy_score(finals_test, finals_pred))

In [ ]:
def getMLPTrainingData():
    i = 0
    lists = []
    labels = []
    maxi = 0
    for h in hyperedges:
        vertex_embedding_list = []
        hyperedge = hyperedges[h]
        lists.append({"h":hyperedge_embeddings[hyperedge_ids.index(h)].tolist(), "f": vertex_features[h].tolist()})
        label = np.zeros((num_categories,))
        label[int(hyperedge["category"])-1] = 1
        labels.append(label)
    X_unshuffled = []
    for hlist in lists:
        x = np.zeros((hyperedge_embedding_dimension + feature_dimension,))
        x[:hyperedge_embedding_dimension] = hlist["h"]
        x[hyperedge_embedding_dimension:] = hlist["f"]
        X_unshuffled.append(x)
    labels = np.asarray(labels)
    X_arr, Y_arr = shuffle(X_unshuffled, labels)
    X_MLP = np.asarray(X_arr)
    Y_MLP = np.asarray(Y_arr)
    return X_MLP, Y_MLP

X_MLP, Y_MLP = getMLPTrainingData()
# np.save(datadir+'/X_MLP_'+str(max_groupsize)+'.npy', X_MLP)
# np.save(datadir+'/Y_MLP_'+str(max_groupsize)+'.npy', Y_MLP)

from sklearn.model_selection import train_test_split
X_MLP_transductive_train, X_MLP_transductive_test, Y_MLP_transductive_train, Y_MLP_transductive_test = train_test_split(X_MLP, Y_MLP, train_size=0.1, test_size=0)

In [ ]:
%load_ext autoreload
%autoreload 2
from keras.callbacks import ModelCheckpoint, TensorBoard
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from Models import MLPTransductive
dataset_name = "cora_transductive"
batch_size = 16
num_epochs = 50

MLP_transductive_model = MLPTransductive(hyperedge_embedding_dimension=hyperedge_embedding_dimension,
                feature_dimension=feature_dimension,
                num_outputs=num_categories,
                dataset_name=dataset_name)
plot_model(MLP_transductive_model, to_file='images/'+dataset_name+'_MLP_transductive_model.png')
checkpointer = ModelCheckpoint(filepath='weights/'+dataset_name+'/MLP_transductive_weights.hdf5', verbose=0, save_best_only=True)
tbCallBack = TensorBoard(log_dir='logs/'+dataset_name+'/MLP_transductive_logs', histogram_freq=0, batch_size=batch_size, write_graph=True, write_grads=True, write_images=True, update_freq='batch')

history = MLP_transductive_model.fit(X_MLP_transductive_train, Y_MLP_transductive_train, epochs=num_epochs, batch_size=batch_size,
        shuffle=True, validation_split=0,
        callbacks=[checkpointer,tbCallBack], verbose=1)

In [ ]:
target_names = ["Neural Networks","Case Based","Reinforcement Learning","Probabilistic Methods","Genetic Algorithms","Rule Learning","Theory"]
from sklearn.metrics import classification_report, accuracy_score

Y_pred_MLP = MLP_transductive_model.predict(X_MLP_transductive_test, batch_size=16, verbose=1)
finals_pred_MLP = []
finals_test_MLP = []
for p in Y_pred_MLP:
    m = 0
    ind = 0
    final = 0
    for i in p:
        if i>m:
            m=i
            final=ind
        ind+=1
    finals_pred_MLP.append(final)

for i in Y_MLP_transductive_test:
    ind=0
    for j in i:
        if j==1:
            finals_test_MLP.append(ind)
        ind+=1
            
print(classification_report(finals_test_MLP, finals_pred_MLP, target_names=target_names,digits=4))
print(accuracy_score(finals_test_MLP, finals_pred_MLP))

In [ ]:
def getDSTrainingData():
    i = 0
    lists = []
    labels = []
    maxi = 0
    for h in hyperedges:
        vertex_embedding_list = []
        hyperedge = hyperedges[h]
        for vertex in hyperedge["members"]:
            i+=1
            if i%100000==0:
                print(i)
            try: # Good ol' nondeterminism
                vertex_embedding_list.append(vertex_embeddings[vertex_ids.index(vertex)].tolist())
            except:
                print("Missed one: ",vertex)
        lists.append({"v":vertex_embedding_list,"f": vertex_features[h].tolist()})
        lists.append
        label = np.zeros((num_categories,))
        label[int(hyperedge["category"])-1] = 1
        labels.append(label)
    X_unshuffled = []
    
    for hlist in lists:
        np_vertex_embeddings = np.asarray(hlist["v"])
        x = np.zeros((vertex_embedding_dimension*max_groupsize + feature_dimension,))
        x[vertex_embedding_dimension*max_groupsize:] = hlist["f"]
        i = 0        
        for embedding in np_vertex_embeddings:
            x[i*embedding.shape[0]:(i+1)*embedding.shape[0]] = embedding
            i+=1
        X_unshuffled.append(x)
    labels = np.asarray(labels)
    X_arr, Y_arr = shuffle(X_unshuffled, labels)
    X = np.asarray(X_arr)
    Y = np.asarray(Y_arr)
    return X, Y

X_deepset, Y_deepset = getDSTrainingData()

print(X_deepset.shape)
print(Y_deepset.shape)

# np.save(datadir+'/X_deepset_'+str(max_groupsize)+'.npy', X_deepset)
# np.save(datadir+'/Y_deepset_'+str(max_groupsize)+'.npy', Y_deepset)

from sklearn.model_selection import train_test_split
X_deepset_transductive_train, X_deepset_transductive_test, Y_deepset_transductive_train, Y_deepset_transductive_test = train_test_split(X_deepset, Y_deepset, train_size=0.2365, test_size=0.7635)

In [ ]:
%load_ext autoreload
%autoreload 2
from keras.callbacks import ModelCheckpoint, TensorBoard
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from Models import DeepSetsTransductive
dataset_name = "cora_transductive"
batch_size = 16
num_epochs = 50

Deepsets_transductive_model = DeepSetsTransductive(vertex_embedding_dimension=vertex_embedding_dimension,
                                                   max_hyperedge_size=max_groupsize,
                                                   feature_dimension=feature_dimension,
                                                   num_outputs=num_categories,
                                                   dataset_name=dataset_name)
plot_model(Deepsets_transductive_model, to_file='images/'+dataset_name+'_Deepsets_transductive_model.png')
checkpointer = ModelCheckpoint(filepath='weights/'+dataset_name+'/Deepsets_transductive_weights.hdf5', verbose=0, save_best_only=True)
tbCallBack = TensorBoard(log_dir='logs/'+dataset_name+'/Deepsets_transductive_logs', histogram_freq=0, batch_size=batch_size, write_graph=True, write_grads=True, write_images=True, update_freq='batch')

history = Deepsets_transductive_model.fit(X_deepset_transductive_train, Y_deepset_transductive_train, epochs=num_epochs, batch_size=batch_size,
        shuffle=True, validation_split=0.78122,
        callbacks=[checkpointer,tbCallBack], verbose=1)

In [ ]:
Y_pred_deepsets = Deepsets_transductive_model.predict(X_deepset_transductive_test, batch_size=16, verbose=1)
finals_pred_deepsets = []
finals_test_deepsets = []
for p in Y_pred_deepsets:
    m = 0
    ind = 0
    final = 0
    for i in p:
        if i>m:
            m=i
            final=ind
        ind+=1
    finals_pred_deepsets.append(final)

for i in Y_deepset_transductive_test:
    ind=0
    for j in i:
        if j==1:
            finals_test_deepsets.append(ind)
        ind+=1
            
print(classification_report(finals_test_deepsets, finals_pred_deepsets, target_names=target_names,digits=4))
print(accuracy_score(finals_test_deepsets, finals_pred_deepsets))

In [ ]:
%load_ext autoreload
%autoreload 2
    
from Models import DeepHyperedgesTransductive
from Models import MLPTransductive
from Models import DeepSetsTransductive

from keras.callbacks import ModelCheckpoint, TensorBoard
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

dataset_name = "cora_transductive"

deephyperedges_transductive_model = DeepHyperedgesTransductive(vertex_embedding_dimension=vertex_embedding_dimension,
                                                                hyperedge_embedding_dimension=hyperedge_embedding_dimension, 
                                                                feature_dimension=feature_dimension,
                                                                max_hyperedge_size=max_groupsize,
                                                                num_outputs=num_categories,
                                                                dataset_name=dataset_name)
deephyperedges_transductive_model.save_weights('models/'+dataset_name+'/deephyperedges_transductive_model.h5')

MLP_transductive_model = MLPTransductive(hyperedge_embedding_dimension=hyperedge_embedding_dimension,
                                            feature_dimension=feature_dimension,
                                            num_outputs=num_categories,
                                            dataset_name=dataset_name)
MLP_transductive_model.save_weights('models/'+dataset_name+'/MLP_transductive_model.h5')

deepsets_transductive_model = DeepSetsTransductive(vertex_embedding_dimension=vertex_embedding_dimension,
                                                       max_hyperedge_size=max_groupsize,
                                                       feature_dimension=feature_dimension,
                                                       num_outputs=num_categories,
                                                       dataset_name=dataset_name)
deepsets_transductive_model.save_weights('models/'+dataset_name+'/deepsets_transductive_model.h5')

In [ ]:
reports = []
batch_size = 16

def hyperedgesTrain(X_train,Y_train,num_epochs):
    deephyperedges_transductive_model.load_weights('models/'+dataset_name+'/deephyperedges_transductive_model.h5')
    history = deephyperedges_transductive_model.fit(X_train, Y_train, epochs=num_epochs, batch_size=batch_size,
            shuffle=True, validation_split=0,
            verbose=0)

def MLPTrain(X_MLP_transductive_train, Y_MLP_transductive_train,num_epochs):
    MLP_transductive_model.load_weights('models/'+dataset_name+'/MLP_transductive_model.h5')
    history = MLP_transductive_model.fit(X_MLP_transductive_train, Y_MLP_transductive_train, epochs=num_epochs, batch_size=batch_size,
            shuffle=True, validation_split=0,
            verbose=0)
    
def DeepSetsTrain(X_deepset_transductive_train,Y_deepset_transductive_train,num_epochs):
    deepsets_transductive_model.load_weights('models/'+dataset_name+'/deepsets_transductive_model.h5')
    history = deepsets_transductive_model.fit(X_deepset_transductive_train, Y_deepset_transductive_train, epochs=num_epochs, batch_size=batch_size,
            shuffle=True, validation_split=0,
            verbose=0)
    
def testModel(model,X_tst,Y_tst):
    from sklearn.metrics import classification_report, accuracy_score
    target_names = ["Neural Networks","Case Based","Reinforcement Learning","Probabilistic Methods","Genetic Algorithms","Rule Learning","Theory"]
    y_pred = model.predict(X_tst, batch_size=16, verbose=0)
    finals_pred = []
    finals_test = []
    for p in y_pred:
        m = 0
        ind = 0
        final = 0
        for i in p:
            if i>m:
                m=i
                final=ind
            ind+=1
        finals_pred.append(final)

    for i in Y_tst:
        ind=0
        for j in i:
            if j==1:
                finals_test.append(ind)
            ind+=1
    c = classification_report(finals_test, finals_pred, target_names=target_names,digits=4)
    reports.append(c)
    print(c)
    
from sklearn.model_selection import train_test_split
def RunAllTests(percentTraining, num_times, num_epochs):
    for i in range(num_times):
        
        print("percent: ",percentTraining,", iteration: ",i+1,", model: deep hyperedges")
        X, Y = getTrainingData()
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=percentTraining, test_size=1-percentTraining)
        hyperedgesTrain(X_train,Y_train,num_epochs)
        testModel(deephyperedges_transductive_model,X_test,Y_test)

#         print("percent: ",percentTraining,", iteration: ",i+1,", model: MLP")
#         X_MLP, Y_MLP = getMLPTrainingData()
#         X_MLP_transductive_train, X_MLP_transductive_test, Y_MLP_transductive_train, Y_MLP_transductive_test = train_test_split(X_MLP, Y_MLP, train_size=percentTraining, test_size=1-percentTraining)
#         MLPTrain(X_MLP_transductive_train, Y_MLP_transductive_train,num_epochs)
#         testModel(MLP_transductive_model,X_MLP_transductive_test,Y_MLP_transductive_test)

#         print("percent: ",percentTraining,", iteration: ",i+1,", model: deep sets")
#         X_deepset, Y_deepset = getDSTrainingData()
#         X_deepset_transductive_train, X_deepset_transductive_test, Y_deepset_transductive_train, Y_deepset_transductive_test = train_test_split(X_deepset, Y_deepset, train_size=percentTraining, test_size=1-percentTraining)
#         DeepSetsTrain(X_deepset_transductive_train,Y_deepset_transductive_train,num_epochs)
#         testModel(deepsets_transductive_model,X_deepset_transductive_test,Y_deepset_transductive_test)

In [ ]:
RunAllTests(0.1, 5, 75)

In [ ]:
RunAllTests(0.3, 5, 75)

In [ ]:
RunAllTests(0.5, 5, 75)

In [ ]:
RunAllTests(0.1, 5, 50)

In [ ]:
RunAllTests(0.3, 5, 50)

In [ ]:
RunAllTests(0.5, 5, 50)

In [ ]:
RunAllTests(0.1, 5, 100)

In [ ]:
RunAllTests(0.3, 5, 100)

In [ ]:
RunAllTests(0.5, 5, 100)